In [28]:
import pandas as pd

In [29]:

# Load CSV
df = pd.read_csv('C:/Users/nazih/Downloads/cic_iot/BenignTraffic.csv')


df['Label'] = 'BenignTraffic' 



# Save updated CSV
df.to_csv('labeled_dataset_benign.csv', index=False)
print(df)

ParserError: Error tokenizing data. C error: out of memory

In [ ]:
# Load CSV
df = pd.read_csv('C:/Users/nazih/Downloads/cic_iot/Spoofing/DNS_Spoofing.csv')

df['Label'] = 'DNS_Spoofing' 



# Save updated CSV
df.to_csv('labeled_dataset_DNS_spoofing.csv', index=False)
print(df)

C:\Users\nazih\AppData\Local\Temp\ipykernel_13620\205077514.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/nazih/Downloads/cic_iot/Spoofing/DNS_Spoofing.csv')


        stream                  src_mac                       dst_mac  \
0           -1  Netatmo Weather Station             ff:ff:ff:ff:ff:ff   
1           -1              Wyze Camera             ff:ff:ff:ff:ff:ff   
2           -1              Wyze Camera             e8:1b:69:f8:d4:e3   
3           -1              Wyze Camera             ff:ff:ff:ff:ff:ff   
4           -1        3c:18:a0:41:c3:a0       Netatmo Weather Station   
...        ...                      ...                           ...   
296020      35        3c:18:a0:41:c3:a0  HeimVision Smart WiFi Camera   
296021      -1        3c:18:a0:41:c3:a0                Netatmo Camera   
296022    4061              Wyze Camera             3c:18:a0:41:c3:a0   
296023      -1  Netatmo Weather Station             ff:ff:ff:ff:ff:ff   
296024    6764        3c:18:a0:41:c3:a0                   Wyze Camera   

                src_ip           dst_ip  src_port  dst_port  \
0                  NaN              NaN         0         0 

In [ ]:
# Load CSV
df = pd.read_csv('C:/Users/nazih/Downloads/cic_iot/Spoofing/MITM-ArpSpoofing.csv')

# Add a label column (manually assign for small datasets)
df['Label'] = 'MITM-ArpSpoofing'

# Save updated CSV
df.to_csv('labeled_dataset_MITM_ArpSpoofing.csv', index=False)
print(df)

C:\Users\nazih\AppData\Local\Temp\ipykernel_13620\1053915760.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/nazih/Downloads/cic_iot/Spoofing/MITM-ArpSpoofing.csv')


        stream             src_mac            dst_mac           src_ip  \
0            0         Wyze Camera  3c:18:a0:41:c3:a0   192.168.137.11   
1            0         Wyze Camera  3c:18:a0:41:c3:a0   192.168.137.11   
2            0         Wyze Camera  3c:18:a0:41:c3:a0   192.168.137.11   
3            0         Wyze Camera  3c:18:a0:41:c3:a0   192.168.137.11   
4            1      Netatmo Camera  3c:18:a0:41:c3:a0   192.168.137.48   
...        ...                 ...                ...              ...   
290432       2  Amazon Echo Studio  3c:18:a0:41:c3:a0  192.168.137.162   
290433       2  Amazon Echo Studio  3c:18:a0:41:c3:a0  192.168.137.162   
290434       2  Amazon Echo Studio  3c:18:a0:41:c3:a0  192.168.137.162   
290435       2  Amazon Echo Studio  3c:18:a0:41:c3:a0  192.168.137.162   
290436       2  Amazon Echo Studio  3c:18:a0:41:c3:a0  192.168.137.162   

                 dst_ip  src_port  dst_port  inter_arrival_time  \
0           35.86.13.17     39807       443 

In [1]:
import pandas as pd

# Load datasets into a dictionary
datasets = {
    'benign': pd.read_csv('labeled_dataset_benign.csv'),
    'dns': pd.read_csv('labeled_dataset_DNS_Spoofing.csv'),
    'mitm': pd.read_csv('labeled_dataset_MITM_ArpSpoofing.csv')
}

# Verify loaded data
for name, df in datasets.items():
    print(f"{name} dataset shape: {df.shape}")
    print(df.head(5))

C:\Users\nazih\AppData\Local\Temp\ipykernel_12336\1652544270.py:6: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  'dns': pd.read_csv('labeled_dataset_DNS_Spoofing.csv'),
C:\Users\nazih\AppData\Local\Temp\ipykernel_12336\1652544270.py:7: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  'mitm': pd.read_csv('labeled_dataset_MITM_ArpSpoofing.csv')


benign dataset shape: (297063, 136)
   stream               src_mac            dst_mac           src_ip  \
0       0  Arlo Q Indoor Camera  3c:18:a0:41:c3:a0  192.168.137.175   
1       0  Arlo Q Indoor Camera  3c:18:a0:41:c3:a0  192.168.137.175   
2       0  Arlo Q Indoor Camera  3c:18:a0:41:c3:a0  192.168.137.175   
3       0  Arlo Q Indoor Camera  3c:18:a0:41:c3:a0  192.168.137.175   
4       0  Arlo Q Indoor Camera  3c:18:a0:41:c3:a0  192.168.137.175   

         dst_ip  src_port  dst_port  inter_arrival_time  \
0  99.81.244.93     56891       443            0.000000   
1  99.81.244.93     56891       443            0.000164   
2  99.81.244.93     56891       443            0.000269   
3  99.81.244.93     56891       443            0.000414   
4  99.81.244.93     56891       443            0.001800   

   time_since_previously_displayed_frame  port_class_dst  ...   min_p   max_p  \
0                               0.000000               1  ...  2908.0  2908.0   
1                   

In [2]:
import pandas as pd

# Load and combine datasets (as before)
datasets = {
    'benign': pd.read_csv('labeled_dataset_benign.csv'),
    'dns': pd.read_csv('labeled_dataset_DNS_Spoofing.csv'),
    'mitm': pd.read_csv('labeled_dataset_MITM_ArpSpoofing.csv')
}
combined_df = pd.concat([datasets['benign'], datasets['dns'], datasets['mitm']], ignore_index=True)
combined_df = combined_df.drop('dst_mac', axis=1)

print(combined_df.shape)


C:\Users\nazih\AppData\Local\Temp\ipykernel_12336\2679432569.py:6: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  'dns': pd.read_csv('labeled_dataset_DNS_Spoofing.csv'),
C:\Users\nazih\AppData\Local\Temp\ipykernel_12336\2679432569.py:7: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  'mitm': pd.read_csv('labeled_dataset_MITM_ArpSpoofing.csv')


(883525, 135)


In [3]:
import re

def is_hex_mac(mac):
    if pd.isna(mac) or mac == 'unknown':  # Skip NaN or filled values
        return False
    pattern = r'^([0-9A-Fa-f]{2}[:-]){5}([0-9A-Fa-f]{2})$'
    return bool(re.match(pattern, str(mac)))

In [4]:
# Remove rows where src_mac is a hex MAC address
combined_df = combined_df[~combined_df['src_mac'].apply(is_hex_mac)]

In [5]:
print(f"Number of duplicate rows: {combined_df.duplicated().sum()}")
combined_df = combined_df.drop_duplicates()
print(f"New shape after removing duplicates: {combined_df.shape}")

Number of duplicate rows: 0
New shape after removing duplicates: (432555, 135)


In [6]:
# Features (X) and targets
X = combined_df.drop(['Label', 'src_mac'], axis=1, errors='ignore')  
y_attack = combined_df['Label']  # For attack detection
y_device = combined_df['src_mac']  # For device identification



In [7]:
print(y_device.unique())

['Arlo Q Indoor Camera' 'HeimVision Smart WiFi Camera'
 'Nest Indoor Camera' 'Teckin Plug 2' 'Amazon Echo Show' 'Yutron Plug 2'
 'GoSund Smart plug WP2 (1)' 'Netatmo Weather Station'
 'Gosund Power strip (1)' 'GoSund Bulb' 'Yi Indoor 2 Camera' 'Wyze Camera'
 'Yutron Plug 1' 'Google Nest Mini Speaker' 'LIFX Lightbulb'
 'HeimVision SmartLife Radio/Lamp' 'Lumiman bulb'
 'GoSund Smart plug WP2 (3)' 'GoSund Power strip (2)' 'Amazon Echo Dot 2'
 'Yi Outdoor Camera' 'Gosund Smart Plug WP3 (2)' 'Atomi Coffee Maker'
 'Amazon Echo Studio' 'GoSund Smart Plug WP2 (2)' 'Teckin Light Strip'
 'Home Eye Camera' 'harman kardon (Ampak Technology)' 'SmartThings Hub'
 'GoSund Smart Plug WP3 (1)' 'Rbcior Camera' 'LampUX RGB'
 'Yi Indoor Camera' 'Teckin Plug 1' 'Netatmo Camera'
 'DCS8000LHA1 D-Link Mini Camera' 'Philips Hue Bridge'
 'AeoTec Smart Home Hub' 'Amazon Echo Dot 1' 'Arlo Base Station'
 'Eufy HomeBase 2' 'Cocoon Smart HVAC Fan' 'iRobot Roomba'
 'AMCREST WiFi Camera' 'Levoit Air Purifier' 'TP-Link 

In [8]:
print(y_attack.value_counts())

Label
MITM-ArpSpoofing    158788
DNS_Spoofing        143031
BenignTraffic       130736
Name: count, dtype: int64


In [9]:


print(X.dtypes)

stream               int64
src_ip              object
dst_ip              object
src_port             int64
dst_port             int64
                    ...   
var_p              float64
q3_p               float64
q1_p               float64
iqr_p              float64
l3_ip_dst_count    float64
Length: 133, dtype: object


In [10]:
categorical_cols = [col for col in X.columns if X[col].nunique() < 10 or X[col].dtype == 'object']
print(f"Categorical features: {categorical_cols}")

Categorical features: ['src_ip', 'dst_ip', 'port_class_dst', 'l4_tcp', 'l4_udp', 'ttl', 'handshake_version', 'handshake_ciphersuites', 'tls_server', 'http_request_method', 'http_host', 'http_response_code', 'user_agent', 'dns_server', 'dns_query_type', 'dns_len_ans', 'device_mac', 'eth_src_oui', 'eth_dst_oui', 'highest_layer', 'http_uri', 'http_content_len', 'http_content_type', 'icmp_type', 'icmp_checksum_status', 'icmp_data_size']


In [11]:
for col in categorical_cols:
    X[col] = X[col].astype(str).fillna('unknown')

#Fill numerical columns with median (or mean)
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
for col in numerical_cols:
    df[col] = df[col].fillna(df[col].median())



In [12]:
from sklearn.preprocessing import LabelEncoder

# Identify categorical columns
categorical_cols = [col for col in X.columns if X[col].nunique() < 10 or X[col].dtype == 'object']
le = LabelEncoder()

for col in categorical_cols:
    X[col] = le.fit_transform(X[col].astype(str).fillna('unknown'))



In [13]:
print(X.dtypes)

stream               int64
src_ip               int32
dst_ip               int32
src_port             int64
dst_port             int64
                    ...   
var_p              float64
q3_p               float64
q1_p               float64
iqr_p              float64
l3_ip_dst_count    float64
Length: 133, dtype: object


In [14]:


le_device = LabelEncoder()
y_device_encoded = le_device.fit_transform(y_device)
print("Encoding completed successfully.")
print(f"Updated X columns: {X.columns.tolist()}")

Encoding completed successfully.
Updated X columns: ['stream', 'src_ip', 'dst_ip', 'src_port', 'dst_port', 'inter_arrival_time', 'time_since_previously_displayed_frame', 'port_class_dst', 'l4_tcp', 'l4_udp', 'ttl', 'eth_size', 'tcp_window_size', 'payload_entropy', 'handshake_version', 'handshake_cipher_suites_length', 'handshake_ciphersuites', 'handshake_extensions_length', 'tls_server', 'handshake_sig_hash_alg_len', 'http_request_method', 'http_host', 'http_response_code', 'user_agent', 'dns_server', 'dns_query_type', 'dns_len_qry', 'dns_interval', 'dns_len_ans', 'device_mac', 'eth_src_oui', 'eth_dst_oui', 'payload_length', 'highest_layer', 'http_uri', 'http_content_len', 'http_content_type', 'icmp_type', 'icmp_checksum_status', 'icmp_data_size', 'jitter', 'stream_1_count', 'stream_1_mean', 'stream_1_var', 'src_ip_1_count', 'src_ip_1_mean', 'src_ip_1_var', 'src_ip_mac_1_count', 'src_ip_mac_1_mean', 'src_ip_mac_1_var', 'channel_1_count', 'channel_1_mean', 'channel_1_var', 'stream_jitte

In [15]:
from sklearn.decomposition import PCA

# Apply PCA to retain 95% of variance (adjust n_components)
X = X.fillna(X.mean()) 

pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X)

print(f"Number of features reduced to: {X_reduced.shape[1]}")
print(f"Explained variance ratio: {sum(pca.explained_variance_ratio_):.2f}")

# Update X (note: PCA output is not labeled, use as numpy array)
X = pd.DataFrame(X_reduced, index=X.index)

Number of features reduced to: 5
Explained variance ratio: 0.95


In [16]:
from sklearn.model_selection import train_test_split

# Split for attack detection
X_train_attack, X_test_attack, y_train_attack, y_test_attack = train_test_split(
    X, y_attack, test_size=0.2, random_state=42, stratify=y_attack
)

# Split for device identification
X_train_device, X_test_device, y_train_device, y_test_device = train_test_split(
    X, y_device_encoded, test_size=0.2, random_state=42, stratify=y_device_encoded
)

In [17]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

In [18]:
# Train attack detection model
rf_model_attack = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, class_weight='balanced')
rf_model_attack.fit(X_train_attack, y_train_attack)

# Train device identification model
rf_model_device = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, class_weight='balanced')
rf_model_device.fit(X_train_device, y_train_device)

RandomForestClassifier(class_weight='balanced', n_jobs=-1, random_state=42)

In [19]:
# Evaluate Random Forest for attack detection
rf_y_pred_attack = rf_model_attack.predict(X_test_attack)
print("Random Forest - Attack Detection Performance:")
print(f"Accuracy: {accuracy_score(y_test_attack, rf_y_pred_attack):.2f}")
print(classification_report(y_test_attack, rf_y_pred_attack))


# Evaluate Random Forest for device identification
rf_y_pred_device = rf_model_device.predict(X_test_device)
print("\nRandom Forest - Device Identification Performance:")
print(f"Accuracy: {accuracy_score(y_test_device, rf_y_pred_device):.2f}")
print(classification_report(y_test_device, rf_y_pred_device, target_names=le_device.classes_))



Random Forest - Attack Detection Performance:
Accuracy: 0.94
                  precision    recall  f1-score   support

   BenignTraffic       0.92      0.92      0.92     26147
    DNS_Spoofing       0.91      0.93      0.92     28606
MITM-ArpSpoofing       0.99      0.98      0.98     31758

        accuracy                           0.94     86511
       macro avg       0.94      0.94      0.94     86511
    weighted avg       0.94      0.94      0.94     86511


Random Forest - Device Identification Performance:
Accuracy: 0.86
                                            precision    recall  f1-score   support

                       AMCREST WiFi Camera       0.65      0.67      0.66       503
                     AeoTec Smart Home Hub       0.63      0.66      0.65       981
                         Amazon Echo Dot 1       0.78      0.65      0.71      1505
                         Amazon Echo Dot 2       0.93      0.96      0.94      5061
                          Amazon Echo Show

In [20]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

In [21]:
# Train AdaBoost for attack detection
ada_model_attack = AdaBoostClassifier(n_estimators=100, random_state=42)
ada_model_attack.fit(X_train_attack, y_train_attack)

# Train AdaBoost for device identification
ada_model_device = AdaBoostClassifier(n_estimators=100, random_state=42)
ada_model_device.fit(X_train_device, y_train_device)

AdaBoostClassifier(n_estimators=100, random_state=42)

In [22]:
# Evaluate AdaBoost for attack detection
ada_y_pred_attack = ada_model_attack.predict(X_test_attack)
print("AdaBoost - Attack Detection Performance:")
print(f"Accuracy: {accuracy_score(y_test_attack, ada_y_pred_attack):.2f}")
print(classification_report(y_test_attack, ada_y_pred_attack))

# Evaluate AdaBoost for device identification
ada_y_pred_device = ada_model_device.predict(X_test_device)
print("\\nAdaBoost - Device Identification Performance:")
print(f"Accuracy: {accuracy_score(y_test_device, ada_y_pred_device):.2f}")
print(classification_report(y_test_device, ada_y_pred_device, target_names=le_device.classes_))

AdaBoost - Attack Detection Performance:
Accuracy: 0.74
                  precision    recall  f1-score   support

   BenignTraffic       0.57      0.81      0.67     26147
    DNS_Spoofing       0.71      0.44      0.54     28606
MITM-ArpSpoofing       0.97      0.96      0.96     31758

        accuracy                           0.74     86511
       macro avg       0.75      0.74      0.72     86511
    weighted avg       0.76      0.74      0.73     86511

\nAdaBoost - Device Identification Performance:
Accuracy: 0.44
                                            precision    recall  f1-score   support

                       AMCREST WiFi Camera       0.00      0.00      0.00       503
                     AeoTec Smart Home Hub       0.00      0.00      0.00       981
                         Amazon Echo Dot 1       0.15      0.02      0.04      1505
                         Amazon Echo Dot 2       0.15      0.93      0.26      5061
                          Amazon Echo Show       0.

c:\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [26]:
!pip install lightgbm

DEPRECATION: Loading egg at c:\python\python311\lib\site-packages\ember-0.1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

In [28]:
# Train LightGBM for attack detection
lgb_model_attack = LGBMClassifier(n_estimators=100, random_state=42, verbose=-1)
lgb_model_attack.fit(X_train_attack, y_train_attack)

# Train LightGBM for device identification
lgb_model_device = LGBMClassifier(n_estimators=100, random_state=42, verbose=-1)
lgb_model_device.fit(X_train_device, y_train_device)

LGBMClassifier(random_state=42, verbose=-1)

In [29]:
# Evaluate LightGBM for attack detection
lgb_y_pred_attack = lgb_model_attack.predict(X_test_attack)
print("LightGBM - Attack Detection Performance:")
print(f"Accuracy: {accuracy_score(y_test_attack, lgb_y_pred_attack):.2f}")
print(classification_report(y_test_attack, lgb_y_pred_attack))

# Evaluate LightGBM for device identification
lgb_y_pred_device = lgb_model_device.predict(X_test_device)
print("\\nLightGBM - Device Identification Performance:")
print(f"Accuracy: {accuracy_score(y_test_device, lgb_y_pred_device):.2f}")
print(classification_report(y_test_device, lgb_y_pred_device, target_names=le_device.classes_))

LightGBM - Attack Detection Performance:
Accuracy: 0.86
                  precision    recall  f1-score   support

   BenignTraffic       0.76      0.82      0.79     26147
    DNS_Spoofing       0.80      0.78      0.79     28606
MITM-ArpSpoofing       0.99      0.96      0.98     31758

        accuracy                           0.86     86511
       macro avg       0.85      0.85      0.85     86511
    weighted avg       0.86      0.86      0.86     86511

\nLightGBM - Device Identification Performance:
Accuracy: 0.31
                                            precision    recall  f1-score   support

                       AMCREST WiFi Camera       0.00      0.00      0.00       503
                     AeoTec Smart Home Hub       0.00      0.00      0.00       981
                         Amazon Echo Dot 1       0.12      0.00      0.01      1505
                         Amazon Echo Dot 2       0.46      0.30      0.37      5061
                          Amazon Echo Show       0.

c:\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [30]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

In [ ]:
# Train Random Forest for attack detection
rf_model_attack = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, class_weight='balanced')
rf_model_attack.fit(X_train_attack, y_train_attack)

# Train LightGBM for attack detection
lgb_model_attack = LGBMClassifier(n_estimators=100, random_state=42, verbose=-1)
lgb_model_attack.fit(X_train_attack, y_train_attack)

# Train Random Forest for device identification
rf_model_device = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, class_weight='balanced')
rf_model_device.fit(X_train_device, y_train_device)

# Train LightGBM for device identification
lgb_model_device = LGBMClassifier(n_estimators=100, random_state=42, verbose=-1)
lgb_model_device.fit(X_train_device, y_train_device)

In [ ]:
# Define hybrid model for attack detection
hybrid_model_attack = VotingClassifier(
    estimators=[('rf', rf_model_attack), ('lgb', lgb_model_attack)],
    voting='soft'  # 'soft' uses predicted probabilities, 'hard' uses majority voting
)
hybrid_model_attack.fit(X_train_attack, y_train_attack)

# Define hybrid model for device identification
hybrid_model_device = VotingClassifier(
    estimators=[('rf', rf_model_device), ('lgb', lgb_model_device)],
    voting='soft'
)
hybrid_model_device.fit(X_train_device, y_train_device)

In [ ]:
# Evaluate hybrid model for attack detection
hybrid_y_pred_attack = hybrid_model_attack.predict(X_test_attack)
print("Hybrid Model - Attack Detection Performance:")
print(f"Accuracy: {accuracy_score(y_test_attack, hybrid_y_pred_attack):.2f}")
print(classification_report(y_test_attack, hybrid_y_pred_attack))

# Evaluate hybrid model for device identification
hybrid_y_pred_device = hybrid_model_device.predict(X_test_device)
print("\\nHybrid Model - Device Identification Performance:")
print(f"Accuracy: {accuracy_score(y_test_device, hybrid_y_pred_device):.2f}")
print(classification_report(y_test_device, hybrid_y_pred_device, target_names=le_device.classes_))